In [8]:
import pandas as pd
import os
import time
import langchain
from typing import List,Dict,Any

from langchain.schema import document
from langchain_core.documents import Document
from langchain.text_splitter import (RecursiveCharacterTextSplitter,CharacterTextSplitter,TokenTextSplitter)
from sqlalchemy.testing.suite.test_reflection import metadata

print("Setup completed")


Setup completed


In [13]:
doc=Document(
    page_content="THis is the main text content that will be embedded and searched!",
    metadata={
        "source":"Example.txt",
        "author":"divyansh",
        "page":1,
        "date_created":"2025-01-01",
        "custom_field":"any_val"
    }
)
print("Document Structure")
print(f"metadata:{doc.metadata}")
print(f"page content:{doc.page_content}")

Document Structure
metadata:{'source': 'Example.txt', 'author': 'divyansh', 'page': 1, 'date_created': '2025-01-01', 'custom_field': 'any_val'}
page content:THis is the main text content that will be embedded and searched!


In [16]:
os.makedirs("data/text_files",exist_ok=True)

In [18]:
sample_texts={
    "data/text_files/python_intro.txt":"""
        python programming language is used for building ai applications
    """
}

for filepath,contents in sample_texts.items():
    with open(filepath,"w",encoding='utf-8') as f:
        f.write(contents)

print("sample texts:")
print(sample_texts)

sample texts:
{'data/text_files/python_intro.txt': '\n        python programming language is used for building ai applications\n    '}


### Text-Loader

In [19]:
from langchain_community.document_loaders import TextLoader
# loading a single text file
loader=TextLoader(file_path='data/text_files/python_intro.txt',encoding='utf-8')

In [23]:
documents = loader.load()

In [38]:
print(documents[0].page_content[1:])
print(documents[0].metadata)
print(len(documents))

        python programming language is used for building ai applications
    
{'source': 'data/text_files/python_intro.txt'}
1


In [47]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # regex to match files
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)

documents=dir_loader.load()

100%|██████████| 2/2 [00:00<00:00, 1001.15it/s]


In [49]:
documents

[Document(metadata={'source': 'data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n'),
 Document(metadata={'source': 'data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming

## Text-splitter Strategies

In [50]:
from langchain.text_splitter import (RecursiveCharacterTextSplitter,CharacterTextSplitter,TokenTextSplitter)

text=documents[0].page_content

In [51]:
text

'Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n'

#### char text splitter

In [53]:
char_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=100,
    chunk_overlap=20,
    length_function=lambda x: len(x)
)

In [54]:
char_chunks=char_splitter.split_text(text)
print(len(char_chunks))

8


In [60]:
char_chunks

['Machine Learning Basics',
 'Machine learning is a subset of artificial intelligence that enables systems to learn and improve',
 'from experience without being explicitly programmed. It focuses on developing computer programs',
 'that can access data and use it to learn for themselves.\nTypes of Machine Learning:',
 '1. Supervised Learning: Learning with labeled data',
 '2. Unsupervised Learning: Finding patterns in unlabeled data',
 '3. Reinforcement Learning: Learning through rewards and penalties',
 'Applications include image recognition, speech processing, and recommendation systems']

### Recursive Character text-splitter

In [69]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n','\n','',' '],
    chunk_size=200,
    chunk_overlap=20,
    length_function=len

)

In [70]:
recursive_text = recursive_text_splitter.split_text(text)

In [71]:
recursive_text

['Machine Learning Basics',
 'Machine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs',
 'that can access data and use it to learn for themselves.',
 'Types of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data',
 '3. Reinforcement Learning: Learning through rewards and penalties',
 'Applications include image recognition, speech processing, and recommendation systems']